## Imports

In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Load in Citizen Science Data

In [2]:
data_path = os.path.join(os.getcwd(), '', 'citizenscience.csv')
citsci = pd.read_csv(data_path, delimiter = ',',  header=None, names=['user_id', 'timestamp'])

In [3]:
citsci['timestamp'] = pd.to_datetime(citsci['timestamp'])

In [4]:
citsci.describe()

,user_id,timestamp
count,99999,99999
unique,943,13370
top,5050a6760454e27aae0026db,2014-12-29 21:38:00
freq,5531,69
first,NaN,2014-12-23 16:53:00
last,NaN,2015-01-05 17:33:00


In [5]:
citsci.dtypes

user_id              object
timestamp    datetime64[ns]
dtype: object

In [6]:
citsci.head()

,user_id,timestamp
0,54999c8d9cd118282b001784,2014-12-23 16:53:00
1,54999c91bb7b56040d0011e8,2014-12-23 16:53:00
2,54999c8fbb7b565d7e000a7c,2014-12-23 16:53:00
3,54999cb29cd1184d77000539,2014-12-23 16:53:00
4,54068fab91ad6b597e000f97,2014-12-23 16:53:00


## Group Records by User_Id

In [7]:
grouped_users = citsci.groupby('user_id')['timestamp'].agg(['first','last'])

In [8]:
grouped_users.head()

,first,last
user_id,,
4d5835cc2a82e278e6000028,2015-01-05 09:07:00,2015-01-05 15:52:00
4fcaa6daae60e44a1900000b,2015-01-01 03:36:00,2015-01-01 03:38:00
50085cfc44dff81137000020,2014-12-28 22:32:00,2015-01-01 20:55:00
5013adf30454e27ae4000002,2015-01-02 22:12:00,2015-01-02 22:12:00
503e2b5eba40af242e000052,2014-12-26 13:12:00,2014-12-29 17:24:00


In [9]:
grouped_users['length'] = grouped_users['last'] - grouped_users['first']

In [23]:
longest_user_id = grouped_users.sort_values('length', ascending=False).iloc[0]
longest_user_id = '508920379177d041ea003fd8'
longest_user = citsci[citsci['user_id'] == '508920379177d041ea003fd8']
longest_user

,user_id,timestamp
12,508920379177d041ea003fd8,2014-12-23 16:53:00
16,508920379177d041ea003fd8,2014-12-23 16:53:00
21,508920379177d041ea003fd8,2014-12-23 16:53:00
44,508920379177d041ea003fd8,2014-12-23 16:54:00
612,508920379177d041ea003fd8,2014-12-23 17:10:00
698,508920379177d041ea003fd8,2014-12-23 17:25:00
7210,508920379177d041ea003fd8,2014-12-24 16:01:00
7211,508920379177d041ea003fd8,2014-12-24 16:01:00
7212,508920379177d041ea003fd8,2014-12-24 16:02:00
7213,508920379177d041ea003fd8,2014-12-24 16:02:00


## Group by User_Id, Timestamp

In [24]:
click_freq = citsci.groupby(['user_id', 'timestamp']).size()

In [25]:
click_freq

user_id                   timestamp          
4d5835cc2a82e278e6000028  2015-01-05 09:07:00     2
                          2015-01-05 09:08:00     1
                          2015-01-05 09:33:00     1
                          2015-01-05 15:04:00     2
                          2015-01-05 15:05:00     1
                          2015-01-05 15:06:00     2
                          2015-01-05 15:07:00     3
                          2015-01-05 15:08:00     3
                          2015-01-05 15:09:00     1
                          2015-01-05 15:16:00     3
                          2015-01-05 15:17:00     4
                          2015-01-05 15:19:00     3
                          2015-01-05 15:20:00     5
                          2015-01-05 15:21:00     4
                          2015-01-05 15:22:00     5
                          2015-01-05 15:23:00     2
                          2015-01-05 15:24:00     3
                          2015-01-05 15:25:00     2
                  